## PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 

Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

```{sql}
SELECT *
FROM `Facilities`
WHERE `membercost` > 0
```

Q2: How many facilities do not charge a fee to members?

```{sql}
SELECT COUNT( `facid` )
FROM `Facilities`
WHERE `membercost` = 0
```

4 facilities do not charge a fee to members.

Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

```{sql}
SELECT `facid` , `name` , `membercost` , `monthlymaintenance`
FROM `Facilities`
WHERE `membercost` !=0 AND `membercost` < .2 * `monthlymaintenance` 
```

Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

```{sql}
SELECT *
FROM `Facilities`
WHERE `facid` IN ( 1, 5 ) 
```

Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than \\$100. Return the name and monthly maintenance of the facilities
in question.

```{sql}
SELECT `name` , `monthlymaintenance` ,
    CASE
    WHEN `monthlymaintenance` > 100 THEN "expnesive"
    ELSE "cheap"
    END AS `cheap_or_expensive`
FROM `Facilities`
```

Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

```{sql}
SELECT `firstname` , `surname`
FROM `Members`
WHERE `joindate` = (SELECT MAX(`joindate`) FROM `Members`) 
```

Darren Smith

Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

```{sql}
SELECT DISTINCT CONCAT( m.firstname, ' ', m.surname ) AS member, f.name
FROM Members AS m
    INNER JOIN Bookings AS b ON b.memid = m.memid
    INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY m.surname
```

Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

```{sql}
SELECT f.name, 
    CASE WHEN b.memid = 0 THEN m.firstname
        ELSE CONCAT( m.firstname, ' ', m.surname ) END AS member,
    CASE WHEN b.memid =0 THEN f.guestcost
        ELSE f.membercost END AS cost
FROM Bookings AS b
    INNER JOIN Facilities AS f ON f.facid = b.facid
    INNER JOIN Members AS m ON m.memid = b.memid
WHERE starttime LIKE '2012-09-14%' AND
    CASE WHEN b.memid = 0 THEN f.guestcost > 30
        ELSE f.membercost > 30 END
ORDER BY cost DESC
```

Q9: This time, produce the same result as in Q8, but using a subquery.

```{sql}
SELECT f.name, 
    CASE WHEN b.memid = 0 THEN m.firstname
        ELSE CONCAT( m.firstname, ' ', m.surname ) END AS member,
    CASE WHEN b.memid =0 THEN f.guestcost
        ELSE f.membercost END AS cost
FROM Bookings AS b
    INNER JOIN Facilities AS f ON f.facid = b.facid
    INNER JOIN Members AS m ON m.memid = b.memid
WHERE starttime LIKE '2012-09-14%' AND
    (SELECT CASE WHEN b.memid = 0 THEN f.guestcost
        ELSE f.membercost END) > 30
ORDER BY cost DESC
```

## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [3]:
query = """
SELECT *
FROM facilities
"""

fac_df = pd.read_sql_query(query ,engine)
fac_df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [4]:
query="""
SELECT *
FROM Bookings
"""

book_df = pd.read_sql_query(query, engine)
book_df.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [5]:
query = """
SELECT *
FROM Members
"""

mem_df = pd.read_sql_query(query, engine)
mem_df.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [6]:
query = """
SELECT f.name as facility,
    SUM(CASE WHEN m.memid = 0 THEN (f.guestcost*b.slots) 
            ELSE (f.membercost*b.slots) END)AS revenue
FROM Bookings AS b
    INNER JOIN Facilities AS f ON f.facid = b.facid
    INNER JOIN Members AS m ON b.memid = m.memid
GROUP BY facility
HAVING revenue < 1000
ORDER BY revenue DESC
"""

q10 = pd.read_sql_query(query, engine)
q10

,facility,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [7]:
query = """
SELECT m1.surname || ', ' ||m1.firstname AS member,
       m2.firstname || ' ' ||m2.surname AS recommender
FROM Members AS m1
    LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid
ORDER BY m1.surname, m1.firstname
"""

q11 = pd.read_sql_query(query, engine)
q11

,member,recommender
0,"Bader, Florence",Ponder Stibbons
1,"Baker, Anne",Ponder Stibbons
2,"Baker, Timothy",Jemima Farrell
3,"Boothe, Tim",Tim Rownam
4,"Butters, Gerald",Darren Smith
5,"Coplin, Joan",Timothy Baker
6,"Crumpet, Erica",Tracy Smith
7,"Dare, Nancy",Janice Joplette
8,"Farrell, David",None
9,"Farrell, Jemima",None


Q12: Find the facilities with their usage by member, but not guests

In [8]:
query = """
SELECT f.name,
       SUM(b.slots) AS usage
FROM Facilities AS f
    INNER JOIN Bookings AS b ON f.facid = b.facid
WHERE b.memid !=0
GROUP BY b.facid
"""

q12 = pd.read_sql_query(query, engine)
q12

,name,usage
0,Tennis Court 1,957
1,Tennis Court 2,882
2,Badminton Court,1086
3,Table Tennis,794
4,Massage Room 1,884
5,Massage Room 2,54
6,Squash Court,418
7,Snooker Table,860
8,Pool Table,856


In [9]:
query = """
SELECT strftime('%m', b.starttime) AS month,
       SUM(b.slots) AS facilities_usage
FROM Facilities AS f
    INNER JOIN Bookings AS b ON f.facid = b.facid
WHERE b.memid !=0
GROUP BY strftime('%m', b.starttime)
"""

q13 = pd.read_sql_query(query, engine)
q13

,month,facilities_usage
0,07,1061
1,08,2531
2,09,3199
